## Introduction


<a id="item31"></a>


## Download and Clean Dataset


Let's start by importing the <em>pandas</em> and the Numpy libraries.


In [1]:
# All Libraries required for this lab are listed below. The libraries pre-installed on Skills Network Labs are commented. 
# If you run this notebook on a different environment, e.g. your desktop, you may need to uncomment and install certain libraries.

#!pip install numpy==1.21.4
#!pip install pandas==1.3.4
#!pip install keras==2.1.6

In [2]:
import pandas as pd
import numpy as np

We will be playing around with the same dataset that we used in the videos.

<strong>The dataset is about the compressive strength of different samples of concrete based on the volumes of the different ingredients that were used to make them. Ingredients include:</strong>

<strong>1. Cement</strong>

<strong>2. Blast Furnace Slag</strong>

<strong>3. Fly Ash</strong>

<strong>4. Water</strong>

<strong>5. Superplasticizer</strong>

<strong>6. Coarse Aggregate</strong>

<strong>7. Fine Aggregate</strong>


## import the dataset


In [3]:
concrete_data = pd.read_csv('concrete_data.csv')
concrete_data.head()

,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate,Age,Strength
0,540.0,0.0,0.0,162.0,2.5,1040.0,676.0,28,79.99
1,540.0,0.0,0.0,162.0,2.5,1055.0,676.0,28,61.89
2,332.5,142.5,0.0,228.0,0.0,932.0,594.0,270,40.27
3,332.5,142.5,0.0,228.0,0.0,932.0,594.0,365,41.05
4,198.6,132.4,0.0,192.0,0.0,978.4,825.5,360,44.30


#### Let's check how many data points we have.


In [4]:
concrete_data.shape

(1030, 9)

So, there are approximately 1000 samples to train our model on. Because of the few samples, we have to be careful not to overfit the training data.


Let's check the dataset for any missing values.


In [5]:
concrete_data.describe()

,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate,Age,Strength
count,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000
mean,281.167864,73.895825,54.188350,181.567282,6.204660,972.918932,773.580485,45.662136,35.817961
std,104.506364,86.279342,63.997004,21.354219,5.973841,77.753954,80.175980,63.169912,16.705742
min,102.000000,0.000000,0.000000,121.800000,0.000000,801.000000,594.000000,1.000000,2.330000
25%,192.375000,0.000000,0.000000,164.900000,0.000000,932.000000,730.950000,7.000000,23.710000
50%,272.900000,22.000000,0.000000,185.000000,6.400000,968.000000,779.500000,28.000000,34.445000
75%,350.000000,142.950000,118.300000,192.000000,10.200000,1029.400000,824.000000,56.000000,46.135000
max,540.000000,359.400000,200.100000,247.000000,32.200000,1145.000000,992.600000,365.000000,82.600000


In [6]:
concrete_data.isnull().sum()

Cement                0
Blast Furnace Slag    0
Fly Ash               0
Water                 0
Superplasticizer      0
Coarse Aggregate      0
Fine Aggregate        0
Age                   0
Strength              0
dtype: int64

The data looks very clean and is ready to be used to build our model.


#### Split data into predictors and target


The target variable in this problem is the concrete sample strength. Therefore, our predictors will be all the other columns.


In [7]:
concrete_data_columns = concrete_data.columns

predictors = concrete_data[concrete_data_columns[concrete_data_columns != 'Strength']] # all columns except Strength
target = concrete_data['Strength'] # Strength column

<a id="item2"></a>


Let's do a quick sanity check of the predictors and the target dataframes.


In [8]:
predictors.head()

,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate,Age
0,540.0,0.0,0.0,162.0,2.5,1040.0,676.0,28
1,540.0,0.0,0.0,162.0,2.5,1055.0,676.0,28
2,332.5,142.5,0.0,228.0,0.0,932.0,594.0,270
3,332.5,142.5,0.0,228.0,0.0,932.0,594.0,365
4,198.6,132.4,0.0,192.0,0.0,978.4,825.5,360


In [9]:
target.head()

0    79.99
1    61.89
2    40.27
3    41.05
4    44.30
Name: Strength, dtype: float64

Finally, the last step is to normalize the data by substracting the mean and dividing by the standard deviation.


In [10]:
predictors_norm = (predictors - predictors.mean()) / predictors.std()
predictors_norm.head()

,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate,Age
0,2.476712,-0.856472,-0.846733,-0.916319,-0.620147,0.862735,-1.217079,-0.279597
1,2.476712,-0.856472,-0.846733,-0.916319,-0.620147,1.055651,-1.217079,-0.279597
2,0.491187,0.795140,-0.846733,2.174405,-1.038638,-0.526262,-2.239829,3.551340
3,0.491187,0.795140,-0.846733,2.174405,-1.038638,-0.526262,-2.239829,5.055221
4,-0.790075,0.678079,-0.846733,0.488555,-1.038638,0.070492,0.647569,4.976069


Let's save the number of predictors to *n_cols* since we will need this number when building our network.


In [11]:
n_cols = predictors_norm.shape[1] # number of predictors

<a id="item1"></a>


<a id='item32'></a>


## Import Keras


Recall from the videos that Keras normally runs on top of a low-level library such as TensorFlow. This means that to be able to use the Keras library, you will have to install TensorFlow first and when you import the Keras library, it will be explicitly displayed what backend was used to install the Keras library. In CC Labs, we used TensorFlow as the backend to install Keras, so it should clearly print that when we import Keras.


#### Let's go ahead and import the Keras library


In [12]:
import keras

As you can see, the TensorFlow backend was used to install the Keras library.


Let's import the rest of the packages from the Keras library that we will need to build our regressoin model.


In [13]:
from keras.models import Sequential
from keras.layers import Dense

<a id='item33'></a>


## Build a Neural Network


Let's define a function that defines our regression model for us so that we can conveniently call it to create our model.


## Task A: Build a Baseline Model

In [14]:
# define regression model
def regression_model():
    # create model
    model = Sequential()
    model.add(Dense(50, activation='relu', input_shape=(n_cols,)))
    model.add(Dense(10, activation='relu'))
    model.add(Dense(1))
    
    # compile model
    model.compile(optimizer='adam', loss='mean_squared_error')
    return model

The above function create a model that has two hidden layers, each of 50 hidden units.


## Train and Test the Network


Let's call the function now to create our model.


In [15]:
# build the model
model = regression_model()

2023-10-06 17:31:13.202774: I tensorflow/core/common_runtime/process_util.cc:146] Creating new thread pool with default inter op setting: 2. Tune using inter_op_parallelism_threads for best performance.


Next, we will train and test the model at the same time using the *fit* method. We will leave out 30% of the data for validation and we will train the model for 100 epochs.


In [16]:
# fit the model
# set to 50 epochs
# reapeat 50 times
MSE = []
for i in range(50):
    model.fit(predictors, target, validation_split=0.3, epochs=50, verbose=2)
    MSE.append(model.evaluate(predictors_norm, target))
print(f' this is the mean of the MSE: {np.array(MSE).mean()} ')
print(f'this is standard diviation of the MSE {np.array(MSE).std()}')

Epoch 1/50
23/23 - 1s - loss: 5092.6895 - val_loss: 203.5681 - 978ms/epoch - 43ms/step
Epoch 2/50
23/23 - 0s - loss: 559.1428 - val_loss: 202.3630 - 89ms/epoch - 4ms/step
Epoch 3/50
23/23 - 0s - loss: 384.4753 - val_loss: 247.9592 - 132ms/epoch - 6ms/step
Epoch 4/50
23/23 - 0s - loss: 351.8267 - val_loss: 188.8325 - 120ms/epoch - 5ms/step
Epoch 5/50
23/23 - 0s - loss: 326.0184 - val_loss: 187.1515 - 118ms/epoch - 5ms/step
Epoch 6/50
23/23 - 0s - loss: 294.5252 - val_loss: 199.3609 - 118ms/epoch - 5ms/step
Epoch 7/50
23/23 - 0s - loss: 273.8170 - val_loss: 171.5842 - 117ms/epoch - 5ms/step
Epoch 8/50
23/23 - 0s - loss: 251.0152 - val_loss: 150.8904 - 124ms/epoch - 5ms/step
Epoch 9/50
23/23 - 0s - loss: 234.7283 - val_loss: 164.2651 - 113ms/epoch - 5ms/step
Epoch 10/50
23/23 - 0s - loss: 223.1612 - val_loss: 155.1521 - 116ms/epoch - 5ms/step
Epoch 11/50
23/23 - 0s - loss: 211.8366 - val_loss: 140.9479 - 146ms/epoch - 6ms/step
Epoch 12/50
23/23 - 0s - loss: 200.4230 - val_loss: 138.9209 -

In [17]:
# what is the mean of the mean squared errors?
# Task A
print(f' this is the mean of the MSE: {np.array(MSE).mean()} ')
print(f'this is standard diviation of the MSE {np.array(MSE).std()}')

 this is the mean of the MSE: 1551.5047412109375 
this is standard diviation of the MSE 11.193823767300241


## Task B: Normalize the data

In [18]:
# check the un normalized data
predictors.head(2)

,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate,Age
0,540.0,0.0,0.0,162.0,2.5,1040.0,676.0,28
1,540.0,0.0,0.0,162.0,2.5,1055.0,676.0,28


In [19]:
# standardize the predictors
predictors_norm = (predictors - predictors.mean()) / predictors.std()
predictors_norm.head(2)

,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate,Age
0,2.476712,-0.856472,-0.846733,-0.916319,-0.620147,0.862735,-1.217079,-0.279597
1,2.476712,-0.856472,-0.846733,-0.916319,-0.620147,1.055651,-1.217079,-0.279597


In [20]:
# using the normalized data
# fit the model
# set to 50 epochs
# reapeat 50 times
MSE = []
for i in range(50):
    model.fit(predictors_norm, target, validation_split=0.3, epochs=50, verbose=2)
    MSE.append(model.evaluate(predictors_norm, target))
print(f' this is the mean of the MSE: {np.array(MSE).mean()} ')
print(f'this is standard diviation of the MSE {np.array(MSE).std()}')

Epoch 1/50
23/23 - 0s - loss: 1558.7524 - val_loss: 905.7368 - 179ms/epoch - 8ms/step
Epoch 2/50


23/23 - 0s - loss: 1061.9840 - val_loss: 438.1176 - 98ms/epoch - 4ms/step
Epoch 3/50
23/23 - 0s - loss: 519.3990 - val_loss: 211.1313 - 84ms/epoch - 4ms/step
Epoch 4/50
23/23 - 0s - loss: 306.0062 - val_loss: 185.2234 - 117ms/epoch - 5ms/step
Epoch 5/50
23/23 - 0s - loss: 251.3962 - val_loss: 183.0783 - 78ms/epoch - 3ms/step
Epoch 6/50
23/23 - 0s - loss: 223.6585 - val_loss: 173.1649 - 124ms/epoch - 5ms/step
Epoch 7/50
23/23 - 0s - loss: 207.0609 - val_loss: 167.2422 - 143ms/epoch - 6ms/step
Epoch 8/50
23/23 - 0s - loss: 194.2158 - val_loss: 162.0898 - 84ms/epoch - 4ms/step
Epoch 9/50
23/23 - 0s - loss: 185.4545 - val_loss: 155.5027 - 89ms/epoch - 4ms/step
Epoch 10/50
23/23 - 0s - loss: 179.6755 - val_loss: 151.2301 - 87ms/epoch - 4ms/step
Epoch 11/50
23/23 - 0s - loss: 172.5376 - val_loss: 145.9244 - 127ms/epoch - 6ms/step
Epoch 12/50
23/23 - 0s - loss: 166.0631 - val_loss: 142.5063 - 140ms/epoch - 6ms/step
Epoch 13/50
23/23 - 0s - loss: 160.8800 - val_loss: 140.9005 - 99ms/epoch - 4m

In [21]:
# how does the mean of the mean squared errors compare to that from Step A?
# Task B
print(f' this is the mean of the MSE: {np.array(MSE).mean()} ')
print(f'this is standard diviation of the MSE {np.array(MSE).std()}')


 this is the mean of the MSE: 61.69835685729981 
this is standard diviation of the MSE 3.8461527483018645


In [28]:
# the mean of the mean squared errors and standard diviation of the MSE is lower in TASK B than in TASK A

## Task C: Increase the number of epochs to 100

In [22]:
# using the normalized data
# fit the model
# set to 100 epochs
# reapeat 100 times
MSE = []
for i in range(50):
    model.fit(predictors_norm, target, validation_split=0.3, epochs=100, verbose=2)
    MSE.append(model.evaluate(predictors_norm, target))
print(f' this is the mean of the MSE: {np.array(MSE).mean()} ')
print(f'this is standard diviation of the MSE {np.array(MSE).std()}')

Epoch 1/100
23/23 - 0s - loss: 7.0851 - val_loss: 176.4977 - 165ms/epoch - 7ms/step
Epoch 2/100
23/23 - 0s - loss: 7.0484 - val_loss: 179.9302 - 118ms/epoch - 5ms/step
Epoch 3/100
23/23 - 0s - loss: 7.1310 - val_loss: 188.6956 - 88ms/epoch - 4ms/step
Epoch 4/100
23/23 - 0s - loss: 6.7845 - val_loss: 189.8643 - 121ms/epoch - 5ms/step
Epoch 5/100
23/23 - 0s - loss: 6.9740 - val_loss: 183.0361 - 75ms/epoch - 3ms/step
Epoch 6/100
23/23 - 0s - loss: 6.9615 - val_loss: 183.2987 - 110ms/epoch - 5ms/step
Epoch 7/100
23/23 - 0s - loss: 6.9605 - val_loss: 190.8596 - 94ms/epoch - 4ms/step
Epoch 8/100
23/23 - 0s - loss: 6.9637 - val_loss: 185.9676 - 94ms/epoch - 4ms/step
Epoch 9/100
23/23 - 0s - loss: 6.8564 - val_loss: 181.0421 - 104ms/epoch - 5ms/step
Epoch 10/100
23/23 - 0s - loss: 6.7652 - val_loss: 184.0969 - 93ms/epoch - 4ms/step
Epoch 11/100
23/23 - 0s - loss: 6.7886 - val_loss: 185.4162 - 95ms/epoch - 4ms/step
Epoch 12/100
23/23 - 0s - loss: 7.0934 - val_loss: 189.8113 - 105ms/epoch - 5ms/

In [23]:
# how does the mean of the mean squared errors compare to that from Step B?
# Task C
print(f' this is the mean of the MSE: {np.array(MSE).mean()} ')
print(f'this is standard diviation of the MSE {np.array(MSE).std()}')

 this is the mean of the MSE: 48.4641837310791 
this is standard diviation of the MSE 3.9039905781228263


In [ ]:
# the mean of the mean squared errors and standard diviation of the MSE is lower in TASK C than in TASK B

## Task D: Increase the number of hidden layers

In [24]:
# define regression model
def regression_model_3_layers():
    # create model
    model = Sequential()
    model.add(Dense(50, activation='relu', input_shape=(n_cols,)))
    model.add(Dense(10, activation='relu'))
    model.add(Dense(10, activation='relu'))
    model.add(Dense(10, activation='relu'))
    model.add(Dense(1))
    
    # compile model
    model.compile(optimizer='adam', loss='mean_squared_error')
    return model

In [25]:
# build the model
model_3_layer = regression_model_3_layers()

In [26]:
# using the normalized data and the 3 layer model
# fit the model
# set to 100 epochs
# reapeat 100 times
MSE = []
for i in range(50):
    model_3_layer.fit(predictors_norm, target, validation_split=0.3, epochs=100, verbose=2)
    MSE.append(model.evaluate(predictors_norm, target))
print(f' this is the mean of the MSE: {np.array(MSE).mean()} ')
print(f'this is standard diviation of the MSE {np.array(MSE).std()}')

Epoch 1/100
23/23 - 1s - loss: 1703.4497 - val_loss: 1224.3187 - 996ms/epoch - 43ms/step
Epoch 2/100
23/23 - 0s - loss: 1658.8372 - val_loss: 1173.8217 - 130ms/epoch - 6ms/step
Epoch 3/100
23/23 - 0s - loss: 1564.6476 - val_loss: 1073.8340 - 164ms/epoch - 7ms/step
Epoch 4/100
23/23 - 0s - loss: 1351.4208 - val_loss: 857.7243 - 91ms/epoch - 4ms/step
Epoch 5/100
23/23 - 0s - loss: 931.9226 - val_loss: 492.6609 - 130ms/epoch - 6ms/step
Epoch 6/100
23/23 - 0s - loss: 436.6498 - val_loss: 193.8149 - 98ms/epoch - 4ms/step
Epoch 7/100
23/23 - 0s - loss: 251.6743 - val_loss: 154.0189 - 155ms/epoch - 7ms/step
Epoch 8/100
23/23 - 0s - loss: 217.2327 - val_loss: 159.5250 - 69ms/epoch - 3ms/step
Epoch 9/100
23/23 - 0s - loss: 201.4467 - val_loss: 156.7617 - 151ms/epoch - 7ms/step
Epoch 10/100
23/23 - 0s - loss: 191.0138 - val_loss: 153.2711 - 87ms/epoch - 4ms/step
Epoch 11/100
23/23 - 0s - loss: 181.4106 - val_loss: 152.6130 - 77ms/epoch - 3ms/step
Epoch 12/100
23/23 - 0s - loss: 174.6222 - val_lo

In [27]:
# how does the mean of the mean squared errors compare to that from Step B?
# Task D
print(f' this is the mean of the MSE: {np.array(MSE).mean()} ')
print(f'this is standard diviation of the MSE {np.array(MSE).std()}')

 this is the mean of the MSE: 44.608646392822266 
this is standard diviation of the MSE 0.0


In [29]:
# the mean of the mean squared errors and standard diviation of the MSE is lower in TASK D than in TASK B

### Thank you for completing this lab!

This notebook was created by [Alex Aklson](https://www.linkedin.com/in/aklson/?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMDeveloperSkillsNetworkDL0101ENSkillsNetwork945-2022-01-01). I hope you found this lab interesting and educational. Feel free to contact me if you have any questions!



## Change Log

|  Date (YYYY-MM-DD) |  Version | Changed By  |  Change Description |
|---|---|---|---|
| 2020-09-21  | 2.0  | Srishti  |  Migrated Lab to Markdown and added to course repo in GitLab |



<hr>

## <h3 align="center"> © IBM Corporation 2020. All rights reserved. <h3/>


This notebook is part of a course on **Coursera** called *Introduction to Deep Learning & Neural Networks with Keras*. If you accessed this notebook outside the course, you can take this course online by clicking [here](https://cocl.us/DL0101EN_Coursera_Week3_LAB1).


<hr>

Copyright &copy; 2019 [IBM Developer Skills Network](https://cognitiveclass.ai/?utm_medium=dswb&utm_source=bducopyrightlink&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMDeveloperSkillsNetworkDL0101ENSkillsNetwork945-2022-01-01&utm_campaign=bdu). This notebook and its source code are released under the terms of the [MIT License](https://bigdatauniversity.com/mit-license/?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMDeveloperSkillsNetworkDL0101ENSkillsNetwork945-2022-01-01).
